# Deploy

In [10]:
import configparser

config = configparser.ConfigParser()
config.read('../config.ini')

username = config['credentials']['username']
password = config['credentials']['password']

bot_name = config['credentials']['bot_name']
bot_pass = config['credentials']['bot_pass']

DEFAULT_HOST_URL = 'https://speakeasy.ifi.uzh.ch'

## Login

In [11]:
from speakeasypy import Speakeasy
speakeasy = Speakeasy(host=DEFAULT_HOST_URL, username=username, password=password)
speakeasy.login()  
# This framework will help you log out automatically when the program terminates.

Login successful. Session token: _TJqsihIRdLLZ2gSSJEDKtTGYFaiFN4d


'_TJqsihIRdLLZ2gSSJEDKtTGYFaiFN4d'

## Chatrooms

In [12]:
# Only check active chatrooms (i.e., remaining_time > 0) if active=True.
rooms = speakeasy.get_rooms(active=True)

In [13]:
rooms

[]

# Query

In [16]:
import rdflib
graph = rdflib.Graph()
graph.parse('../data/14_graph.nt', format='turtle')

query_string = """# Who directed the movie Apocalypse Now?  

PREFIX ddis: <http://ddis.ch/atai/>   
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX schema: <http://schema.org/>   

SELECT ?director WHERE {  
    ?movie rdfs:label "Apocalypse Now"@en .  
        ?movie wdt:P57 ?directorItem . 
    ?directorItem rdfs:label ?director . 
}  

LIMIT 1  """

[str(s) for s, in graph.query(query_string)]

['Francis Ford Coppola']

In [15]:
# top user-rated movies
[str(s) for s, in graph.query('''
    PREFIX ddis: <http://ddis.ch/atai/> 
PREFIX ddis: <http://ddis.ch/atai/>   
PREFIX wd: <http://www.wikidata.org/entity/>   
PREFIX wdt: <http://www.wikidata.org/prop/direct/>   
PREFIX schema: <http://schema.org/>   

SELECT ?lbl WHERE {  
    ?movie wdt:P31 wd:Q11424 .  
    ?movie ddis:rating ?rating .  
    ?movie rdfs:label ?lbl .  
}  

ORDER BY DESC(?rating)   
LIMIT 1 
    ''')]

['Forrest Gump']

## Agent

In [14]:
from speakeasypy import Chatroom
from typing import List
import time

DEFAULT_HOST_URL = 'https://speakeasy.ifi.uzh.ch'
listen_freq = 2


class Agent:
    def __init__(self, username, password):
        self.username = username
        # Initialize the Speakeasy Python framework and login.
        self.speakeasy = Speakeasy(host=DEFAULT_HOST_URL, username=username, password=password)
        self.speakeasy.login()  # This framework will help you log out automatically when the program terminates.

    def listen(self):
        while True:
            # only check active chatrooms (i.e., remaining_time > 0) if active=True.
            rooms: List[Chatroom] = self.speakeasy.get_rooms(active=True)
            for room in rooms:
                if not room.initiated:
                    # send a welcome message if room is not initiated
                    room.post_messages(f'Hello! This is a welcome message from {room.my_alias}.')
                    room.initiated = True
                # Retrieve messages from this chat room.
                # If only_partner=True, it filters out messages sent by the current bot.
                # If only_new=True, it filters out messages that have already been marked as processed.
                for message in room.get_messages(only_partner=True, only_new=True):
                    print(
                        f"\t- Chatroom {room.room_id} "
                        f"- new message #{message.ordinal}: '{message.message}' "
                        f"- {self.get_time()}")

                    # Implement your agent here #
                    # top user-rated movies

                    # Send a message to the corresponding chat room using the post_messages method of the room object.
                    room.post_messages(f"Received your message: '{message.message}' ")
                    # Mark the message as processed, so it will be filtered out when retrieving new messages.
                    room.mark_as_processed(message)

                # Retrieve reactions from this chat room.
                # If only_new=True, it filters out reactions that have already been marked as processed.
                for reaction in room.get_reactions(only_new=True):
                    print(
                        f"\t- Chatroom {room.room_id} "
                        f"- new reaction #{reaction.message_ordinal}: '{reaction.type}' "
                        f"- {self.get_time()}")

                    # Implement your agent here #

                    room.post_messages(f"Received your reaction: '{reaction.type}' ")
                    room.mark_as_processed(reaction)

            time.sleep(listen_freq)

    @staticmethod
    def get_time():
        return time.strftime("%H:%M:%S, %d-%m-%Y", time.localtime())


if __name__ == '__main__':
    demo_bot = Agent(bot_name, bot_pass)
    demo_bot.listen()


Login successful. Session token: 4Jf5mEfPTN_mQSnyr2diXUQuyDZ_lybJ
	- Chatroom 05910dd3-ad3a-41d1-992a-196979290da1 - new message #1: 'PREFIX ddis: <http://ddis.ch/atai/>   
PREFIX wd: <http://www.wikidata.org/entity/>   
PREFIX wdt: <http://www.wikidata.org/prop/direct/>   
PREFIX schema: <http://schema.org/>   

SELECT ?lbl WHERE {  
    ?movie wdt:P31 wd:Q11424 .  
    ?movie ddis:rating ?rating .  
    ?movie rdfs:label ?lbl .  
}  

ORDER BY DESC(?rating)   
LIMIT 1 ' - 13:44:46, 02-10-2024


KeyboardInterrupt: 

# Data